# Experiments

## Overview
This notebook contains preliminary experiments comparing different AI agent configurations. Each scenario will use identical study material input.

## Scenarios
There will be 4 scenarios:
1. **Single agent 0-shot**: One agent with no examples provided
2. **Single agent 1-shot**: One agent with one example provided
3. **Multi-agent 0-shot**: Two agents (question generator and evaluator) with no examples, using manual agent orchestration
4. **Multi-agent 1-shot**: Two agents with one example, using manual agent orchestration

## Methodology
- Each scenario will have same study material input
- Each scenario will be run once as this is a preliminary study
- The multi-agent scenarios will utilize the crewAI framework
- Results will be compared qualitatively rather than statistically


In [1]:
%pip install google-genai


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Base Setup

In [2]:
import os
from google import genai
from google.genai import types
import json

In [3]:
base_model = "gemini-2.0-flash"
api_key = os.environ.get("GEMINI_API_KEY")
client = genai.Client(
  api_key=api_key,
)

# setup study material files
files = [
  client.files.upload(file='section-loop.pdf')
]

## Scenario 1 & 2 Setup

In [32]:
question_generator_prompt = ""
with open('question-generator-prompt.txt', 'r') as file:
    question_generator_prompt = file.read()

def setup_question_generator_config(system_prompt):
    return types.GenerateContentConfig(
        response_mime_type="application/json",
        system_instruction=[
            types.Part.from_text(text=system_prompt),
        ],
    )

question_generator_config_0_shot = setup_question_generator_config(question_generator_prompt)

## Scenario 1

In [33]:
contents_scenario_1 = [
    types.Content(
        role="user",
        parts=[
            types.Part.from_uri(
                file_uri=files[0].uri,
                mime_type=files[0].mime_type,
            ),
            types.Part.from_text(text="Generate exactly 5 MCQs covering the different loop concepts and examples presented **in the provided file**. Use the JSON format specified in your instructions."),
        ]
    )
]

In [34]:
result = client.models.generate_content(
  model=base_model,
  contents=contents_scenario_1,
  config=question_generator_config_0_shot,
)

In [35]:
print(f"{result.text}")

[
  {
    "question_text": "What is the primary difference between a `while` loop and a `for` loop, as described in the text?",
    "options": {
      "A": "A `while` loop iterates over a container, while a `for` loop executes as long as a condition is true.",
      "B": "A `for` loop iterates over a container, while a `while` loop executes as long as a condition is true.",
      "C": "A `while` loop can only be used with numerical data, while a `for` loop can only be used with strings.",
      "D": "There is no difference; `while` and `for` loops are interchangeable."
    },
    "correct_option": "B"
  },
  {
    "question_text": "In a `while` loop, when does the loop expression get evaluated?",
    "options": {
      "A": "Only after the entire loop body has executed.",
      "B": "Only before the loop body executes for the first time.",
      "C": "Before each iteration and after the loop statement is executed.",
      "D": "Only when a `break` statement is encountered."
    },
    

## Scenario 2

In [36]:
scenario_2_prompt = ""
with open('scenario-2-prompts.txt', 'r') as file:
    scenario_2_prompt = file.read()

question_generator_config_1_shot = setup_question_generator_config(scenario_2_prompt)

In [37]:
contents_scenario_2 = [
    types.Content(
        role="user",
        parts=[
            types.Part.from_uri(
                file_uri=files[0].uri,
                mime_type=files[0].mime_type,
            ),
            types.Part.from_text(text="""Here is an example of the desired output format:
Example Output:
[
  {
    "question_text": "What is the output of the code snippet?\n`count = 0\nwhile count < 3:\n  print(count)\n  count += 1`",
    "options": {
      "A": "0 1 2 3",
      "B": "1 2 3",
      "C": "0 1 2",
      "D": "0 1"
    },
    "correct_option": "C"
  }
]

Now, using that exact JSON format, generate exactly 5 MCQs covering the different loop concepts and examples presented **in the provided file**.
"""),
        ]
    )
]

In [38]:
result_1_shot = client.models.generate_content(
  model=base_model,
  contents=contents_scenario_2,
  config=question_generator_config_1_shot,
)

In [39]:
print(f"{result_1_shot.text}")

[
  {
    "question_text": "Which of the following statements is true about a 'while' loop?",
    "options": {
      "A": "The loop body executes only once.",
      "B": "The loop body executes as long as the loop expression is false.",
      "C": "The loop body executes as long as the loop expression is true.",
      "D": "The loop expression is evaluated only after the loop body is executed."
    },
    "correct_option": "C"
  },
  {
    "question_text": "What is the purpose of the 'break' statement in a loop?",
    "options": {
      "A": "To skip the current iteration and proceed to the next.",
      "B": "To exit the loop prematurely.",
      "C": "To execute the 'else' block of the loop.",
      "D": "To restart the loop from the beginning."
    },
    "correct_option": "B"
  },
  {
    "question_text": "What is the output of the following code?\n`for i in range(2, 8, 2):\n    print(i)`",
    "options": {
      "A": "2 3 4 5 6 7",
      "B": "2 4 6 8",
      "C": "2 4 6",
      "

## Scenario 3 & 4 Setup

In [40]:
content_with_initial_mcqs_1 = contents_scenario_1
content_with_initial_mcqs_2 = contents_scenario_2

content_with_initial_mcqs_1.append(
  types.Content(
    role="model",
    parts=[
      types.Part.from_text(text=f"""Evaluate the following 5 generated MCQs based on the criteria specified in your instructions (Topic Coverage/Relevance, Question Quality/Clarity, Answer Quality/Distractors, Correctness Verification - scale 1-5). 
Provide the scores, a brief comment for each question, and an overall topic coverage comment. 
Use ONLY the specified JSON output format. 
Generated MCQs: {result.text}
"""),
    ]
  )
)

content_with_initial_mcqs_2.append(
  types.Content(
    role="model",
    parts=[
      types.Part.from_text(text=f"""Here is an example of the desired JSON evaluation output format:
Example Evaluation Output:
{{
  'evaluation_results': [
    {{
      'question_evaluated': 'What does the `break` statement do inside a loop?',
      'evaluation': {{
        'topic_coverage_relevance_score': 5,
        'question_quality_clarity_score': 5,
        'answer_quality_distractors_score': 4,
        'correctness_verification_score': 5,
        'brief_comment': 'Tests fundamental loop control. Clear question. Distractors plausible.'
      }}
    }}
  ],
  'overall_topic_coverage_comment': 'The single example question covers loop control well, but a full set would need to cover loop types too.'
}}

Now, using that exact JSON format, evaluate the following 5 generated MCQs based on the criteria specified in your instructions (Topic Coverage/Relevance, Question Quality/Clarity, Answer Quality/Distractors, Correctness Verification - scale 1-5). Provide the scores, a brief comment for each question, and an overall topic coverage comment.

Generated MCQs: {result_1_shot.text}"""),
    ]
  )
)

## Scenario 3

In [41]:
evaluator_0_shot_prompt = ""
with open('evaluator-0-shot-prompt.txt', 'r') as file: 
    evaluator_0_shot_prompt = file.read()

content_with_initial_mcqs_1.append(
    types.Content(
        role="user",
        parts=[
            types.Part.from_text(text="Evaluate the following MCQs based on the context."),
        ]
    )
)

evaluator_agent_config_1 = types.GenerateContentConfig(
    response_mime_type="application/json",
    system_instruction=[
        types.Part.from_text(text=evaluator_0_shot_prompt),
    ],
)

In [42]:
evaluator_result_1 = client.models.generate_content(
  model=base_model,
  contents=content_with_initial_mcqs_1,
  config=evaluator_agent_config_1,
)

In [43]:
print(f">> evaluator_result_1: {evaluator_result_1.text}")

>> evaluator_result_1: {
  "evaluation_results": [
    {
      "question_evaluated": "What is the primary difference between a `while` loop and a `for` loop, as described in the text?",
      "evaluation": {
        "topic_coverage_relevance_score": 5,
        "question_quality_clarity_score": 5,
        "answer_quality_distractors_score": 5,
        "correctness_verification_score": 5,
        "brief_comment": "This question effectively covers the fundamental difference between `for` and `while` loops, using clear language and plausible distractors."
      }
    },
    {
      "question_evaluated": "In a `while` loop, when does the loop expression get evaluated?",
      "evaluation": {
        "topic_coverage_relevance_score": 5,
        "question_quality_clarity_score": 5,
        "answer_quality_distractors_score": 5,
        "correctness_verification_score": 5,
        "brief_comment": "This question accurately addresses the timing of `while` loop condition evaluation, with reasona

In [44]:
content_with_initial_mcqs_1.append(
  types.Content(
    role="model",
    parts=[
      types.Part.from_text(text=f"Feedback from evaluator: {evaluator_result_1.text}"),
    ]
  ),
)
content_with_initial_mcqs_1.append(
  types.Content(
    role="user",
    parts=[
      types.Part.from_text(text="Regenerate MCQs based on the feedback.")
    ]
  )
)

TypeError: list.append() takes exactly one argument (2 given)

In [20]:
# Send feedback to question generator
result_with_feedback_1 = client.models.generate_content(
  model=base_model,
  contents=content_with_initial_mcqs_1,
  config=question_generator_config_0_shot,
)

In [21]:
print(f">> result_with_feedback_1: {result_with_feedback_1.text}")

>> result_with_feedback_1: [
  {
    "question_text": "What is the primary difference between a `while` loop and a `for` loop as introduced in the text?",
    "options": {
      "A": "A `while` loop iterates over a container, while a `for` loop executes a block of code a fixed number of times.",
      "B": "A `while` loop continues as long as a condition is true, while a `for` loop iterates over elements in a container or a sequence.",
      "C": "A `while` loop requires a `break` statement to terminate, while a `for` loop automatically terminates after processing all elements.",
      "D": "A `while` loop is used for numerical calculations, while a `for` loop is used for string manipulation."
    },
    "correct_option": "B"
  },
  {
    "question_text": "Consider the following code snippet:\n\ncounter = 1\nwhile counter <= 5:\n    print(counter)\n    if counter == 3:\n        break\n    counter += 1\n\nWhat will be the output of this code?",
    "options": {
      "A": "1 2 3 4 5",
 

## Scenario 4

In [22]:
evaluator_1_shot_prompt = ""
with open('evaluator-1-shot-prompt.txt', 'r') as file: 
    evaluator_1_shot_prompt = file.read()

content_with_initial_mcqs_2.append(
    types.Content(
        role="user",
        parts=[
            types.Part.from_text(text="Evaluate the following MCQs based on the context."),
        ]
    )
)

evaluator_agent_config_2 = types.GenerateContentConfig(
    response_mime_type="application/json",
    system_instruction=[
        types.Part.from_text(text=evaluator_1_shot_prompt),
    ],
)

In [23]:
evaluator_result_2 = client.models.generate_content(
  model=base_model,
  contents=content_with_initial_mcqs_2,
  config=evaluator_agent_config_2,
)

In [24]:
print(f">> evaluator_result_2: {evaluator_result_2.text}")

>> evaluator_result_2: {
  "evaluation_results": [
    {
      "question_evaluated": "Which of the following statements is true about a `while` loop?",
      "evaluation": {
        "topic_coverage_relevance_score": 5,
        "question_quality_clarity_score": 5,
        "answer_quality_distractors_score": 4,
        "correctness_verification_score": 5,
        "brief_comment": "Covers the fundamental behavior of while loops. The question is clear, and the distractors represent common misconceptions."
      }
    },
    {
      "question_evaluated": "What is a 'nested loop'?",
      "evaluation": {
        "topic_coverage_relevance_score": 5,
        "question_quality_clarity_score": 5,
        "answer_quality_distractors_score": 5,
        "correctness_verification_score": 5,
        "brief_comment": "Clearly defines nested loops. All distractors are plausible, making this a good question."
      }
    },
    {
      "question_evaluated": "What does the `break` statement do in a loop?

In [25]:
content_with_initial_mcqs_2.append(
  types.Content(
    role="model",
    parts=[
      types.Part.from_text(text=f"Feedback from evaluator: {evaluator_result_2.text}")
    ]
  ),
)

content_with_initial_mcqs_2.append(
  types.Content(
    role="user",
    parts=[
      types.Part.from_text(text="Regenerate MCQs based on the feedback.")
    ]
  )
)

In [31]:
print(">> Content with initial MCQs 2:")
for content in content_with_initial_mcqs_2:
    print(f"\nRole: {content.role}")
    for part in content.parts:
        print(f"Part text: {part.text}")
    print("-" * 50)

>> Content with initial MCQs 2:

Role: user
Part text: None
Part text: Here is an example of the desired output format:
Example Output:
[
  {
    "question_text": "What is the output of the code snippet?
`count = 0
while count < 3:
  print(count)
  count += 1`",
    "options": {
      "A": "0 1 2 3",
      "B": "1 2 3",
      "C": "0 1 2",
      "D": "0 1"
    },
    "correct_option": "C"
  }
]

Now, using that exact JSON format, generate exactly 5 MCQs covering the different loop concepts and examples presented **in the provided file**.

--------------------------------------------------

Role: model
Part text: Here is an example of the desired JSON evaluation output format:
Example Evaluation Output:
{
  'evaluation_results': [
    {
      'question_evaluated': 'What does the `break` statement do inside a loop?',
      'evaluation': {
        'topic_coverage_relevance_score': 5,
        'question_quality_clarity_score': 5,
        'answer_quality_distractors_score': 4,
        'corre

In [28]:
result_after_feedback_2 = client.models.generate_content(
  model=base_model,
  contents=content_with_initial_mcqs_2,
  config=question_generator_config_1_shot,
)

In [29]:
print(f">> result_after_feedback_2: {result_after_feedback_2.text}")

>> result_after_feedback_2: {
  "evaluation_results": [
    {
      "question_evaluated": "Which of the following statements is true about a `while` loop?",
      "evaluation": {
        "topic_coverage_relevance_score": 5,
        "question_quality_clarity_score": 5,
        "answer_quality_distractors_score": 4,
        "correctness_verification_score": 5,
        "brief_comment": "Covers the fundamental behavior of while loops. The question is clear, and the distractors represent common misconceptions."
      }
    },
    {
      "question_evaluated": "What is a 'nested loop'?",
      "evaluation": {
        "topic_coverage_relevance_score": 5,
        "question_quality_clarity_score": 5,
        "answer_quality_distractors_score": 5,
        "correctness_verification_score": 5,
        "brief_comment": "Clearly defines nested loops. All distractors are plausible, making this a good question."
      }
    },
    {
      "question_evaluated": "What does the `break` statement do in a 